### Data Loading

In [1]:
!rm -r data
import os
try:
  os.mkdir('./data')
except:
  pass

!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data

rm: cannot remove 'data': No such file or directory
--2020-09-14 11:50:12--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./data/input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2020-09-14 11:50:13 (23.8 MB/s) - ‘./data/input.txt’ saved [1115394/1115394]



In [2]:
import torch
import torch.nn as nn

In [3]:
!pip install unidecode

     |████████████████████████████████| 245kB 4.3MB/s 


In [4]:
import unidecode
import string
import random
import re
import time,math

In [5]:
num_epochs = 2000
print_every = 100
plot_every = 10
chunk_len = 200
hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [6]:
all_characters = string.printable # 출력 가능한 모든 문자 불러오기
n_characters = len(all_characters)
print(all_characters)
print(n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

100


In [7]:
file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print('file_len = ',file_len)

file_len =  1115394


In [9]:
# txt 파일의 일부분을 랜덤하게 불러오기
def random_chunk():
  start_index = random.randint(0,file_len - chunk_len) # 시작 지점 설정
  end_index = start_index + chunk_len +1
  return file[start_index:end_index]

print(random_chunk())

 I have to love thee
Doth much excuse the appertaining rage
To such a greeting: villain am I none;
Therefore farewell; I see thou know'st me not.

TYBALT:
Boy, this shall not excuse the injuries
That t


In [10]:
# ch -> tensor
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
    tensor[c] = all_characters.index(string[c])
  return tensor
print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [11]:
# 랜덤한 텍스트를 불러와서 입력값과 목표값으로 바꿔주는 함수
def random_training_set():
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp,target

In [21]:
class RNN(nn.Module):
  def __init__(self,input_size,embedding_size,hidden_size,output_size,num_layers = 1):
    super(RNN,self).__init__()
    self.input_size = input_size
    self.embedding_size = embedding_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    
    self.encoder = nn.Embedding(self.input_size,self.embedding_size)
    self.rnn = nn.RNN(self.embedding_size,self.hidden_size,self.num_layers)
    self.decoder = nn.Linear(self.hidden_size,self.output_size)
  
  def forward(self,input,hidden):
    out = self.encoder(input.view(1,-1))
    out,hidden = self.rnn(out,hidden)
    out = self.decoder(out.view(batch_size,-1))
    return out,hidden
  
  def init_hidden(self):
    hidden = torch.zeros(self.num_layers,batch_size,self.hidden_size)
    return hidden
  
model = RNN(n_characters,embedding_size,hidden_size,n_characters,num_layers)

In [22]:
model = RNN(input_size = n_characters,
            embedding_size = embedding_size,
            hidden_size = hidden_size,
            output_size = n_characters,
            num_layers = 2)

In [23]:
# test
inp = char_tensor('A')
print(inp)
hidden = model.init_hidden()
print(hidden.size())
out,hidden = model(inp,hidden)
print(out.size())

tensor([36])
torch.Size([2, 1, 100])
torch.Size([1, 100])


In [24]:
optimizer = torch.optim.Adam(model.parameters(),lr = lr)
loss_func = nn.CrossEntropyLoss()

In [27]:
def test():
  start_str = 'b'
  inp = char_tensor(start_str)
  hidden = model.init_hidden()
  x = inp

  print(start_str,end = '')
  for i in range(200):
    output,hidden = model(x,hidden)
    output_dist = output.data.view(-1).div(0.8).exp()
    top_i = torch.multinomial(output_dist,1)[0] # 높은 값을 가지는 문자들 중 랜덤하게 추출
    predicted_char = all_characters[top_i]

    print(predicted_char,end='')

    x= char_tensor(predicted_char)

In [28]:
# train
for i in range(num_epochs):
  inp,label = random_training_set()
  hidden = model.init_hidden()
  
  loss = torch.tensor([0]).type(torch.FloatTensor)
  optimizer.zero_grad()
  for j in range(chunk_len-1):
    x = inp[j]
    y_ = label[j].unsqueeze(0).type(torch.LongTensor)
    y,hidden = model(x,hidden)
    loss +=loss_func(y,y_)
  
  loss.backward()
  optimizer.step()

  if i % 500 == 0:
    print('\n',loss/chunk_len,'\n')
    test()
    print('\n','='*100)


 tensor([4.4868], grad_fn=<DivBackward0>) 

0hT}fEe	3Bx[ZRl)
v+&r>B~u;|U7o^.k(o5d6kCz-._9DQf/lX2<QPFQ{Hm(9.q8i9#w6|X
u

 tensor([2.0163], grad_fn=<DivBackward0>) 

ble to brow ot row a bejoy of to were pour porben, this bland, shy will gove mans arme heese overy hone, blow, of have bowe wimy of doin, we est
And in that but themes to so oxpwight ess broming bast s

 tensor([1.7002], grad_fn=<DivBackward0>) 

bod must the it made my master me of made por mide souper
And strobe the ore preavind that her think beed ente, go me my bine of my me moth with be the bone moward the thible her
God is that to make mu

 tensor([1.6802], grad_fn=<DivBackward0>) 

banth!

ANGLLO:
What dock agoth the gresert! Reives seaves fat his upon my lord in you:

RORCI:
Hoth,--
As not out sean?

MARIA:
I whow thy their love speak sulfove beas this and that are fathing a dre
